In [7]:
# necessary packages
import pandas as pd

## Import the dataset

In [8]:
df_city = pd.read_csv('/content/geonames-all-cities-with-a-population-1000.csv',delimiter=';')
df_city.head()

,Geoname ID,Name,ASCII Name,Alternate Names,Feature Class,Feature Code,Country Code,Country name EN,Country Code 2,Admin1 Code,Admin2 Code,Admin3 Code,Admin4 Code,Population,Elevation,DIgital Elevation Model,Timezone,Modification date,LABEL EN,Coordinates
0,2744042,Zandvoort,Zandvoort,"Landvoort,Zandvoort,Зандвоорт",P,PPL,NL,Netherlands,NaN,07,0473,NaN,NaN,16868,NaN,6,Europe/Amsterdam,2018-05-08,Netherlands,"52.37125, 4.53306"
1,2744424,Wilhelminadorp,Wilhelminadorp,NaN,P,PPLX,NL,Netherlands,NaN,06,0753,NaN,NaN,3645,NaN,17,Europe/Amsterdam,2017-03-24,Netherlands,"51.49742, 5.39223"
2,2744459,Wijnjewoude,Wijnjewoude,"Duurswoude,Wijnjeterp,Wijnjeterp-Duurswoude,Wi...",P,PPL,NL,Netherlands,NaN,02,0086,NaN,NaN,1245,NaN,4,Europe/Amsterdam,2017-10-17,Netherlands,"53.05814, 6.2047"
3,2745271,Voorthuizen,Voorthuizen,NaN,P,PPL,NL,Netherlands,NaN,03,0203,NaN,NaN,9710,NaN,15,Europe/Amsterdam,2017-10-17,Netherlands,"52.18667, 5.60556"
4,2746228,Tolkamer,Tolkamer,"Tokalmar,Tolkmar",P,PPL,NL,Netherlands,NaN,03,0299,NaN,NaN,2430,NaN,14,Europe/Amsterdam,2019-01-20,Netherlands,"51.855, 6.10278"


## Function of filtering the countries that don't host a megapoliss

In [9]:
def country_without_megapoliss(dataframe):
  # group by country code and calculate the maximum population
  df_max_population = dataframe.groupby('Country Code')['Population'].max()
  # filter the country's maximum population of all the cities that lower then 10,000,000
  df_code = pd.DataFrame(df_max_population[df_max_population < 10000000].index)
  return df_code

## Using the open data to merge the country name without NULL value

In [10]:
# read the csv containing the full country code and country name
full_country_name = pd.read_excel('/content/iso_2digit_alpha_country_codes.xls', header=2)

# rename the column
full_country_name = full_country_name.rename(columns={'Code Value': 'Country Code', 'Definition': 'Country name EN'})
full_country_name.head()

,Country Code,Country name EN
0,AF,Afghanistan
1,AX,Aland Islands
2,AL,Albania
3,DZ,Algeria
4,AS,American Samoa


In [11]:
# get the country code we needed
try:
  df_code = country_without_megapoliss(df_city)
except:
  print('Check the column name and data type ofinput dataframe')

# merge the country name in the filter dataframe
df_result = df_code.merge(full_country_name[['Country Code','Country name EN']], on='Country Code', how='left').drop_duplicates(subset='Country Code')

# order by country name
result = df_result.sort_values('Country name EN').reset_index(drop=True)
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Country Code     235 non-null    object
 1   Country name EN  235 non-null    object
dtypes: object(2)
memory usage: 3.8+ KB


## Save it as the tsv file

In [12]:
result.to_csv('result.tsv', sep='\t', index=False)